In [1]:
import re
import nltk
# import cudf.pandas
# cudf.pandas.install()
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

lemmatizer = nltk.stem.WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer

# Load Data

In [2]:
df_cleantech_keywords = pd.read_json("/home/thiesen/Documents/Cleantech_Concepts/cleantech_keywords_similarity_015_co_occurrence_025_claim_fulltext.json")

In [3]:
df_patstat = pd.read_json('/mnt/hdd01/PATSTAT Working Directory/PATSTAT/df_patstat_cleantech_granted_abstract_metadata.json')

In [4]:
df_patstat = df_patstat[df_patstat['appln_auth'].apply(lambda x: 'US' in x or 'EP' in x)]

In [5]:
df_patstat['appln_title'] = df_patstat['appln_title'].progress_apply(lambda x: ' '.join([i if i is not None else '' for i in x])).str.lower()
df_patstat['appln_abstract'] = df_patstat['appln_abstract'].progress_apply(lambda x: ' '.join([i if i is not None else '' for i in x])).str.lower()
df_patstat['title_abstract'] = df_patstat['appln_title'] + ' ' + df_patstat['appln_abstract']
df_patstat['title_abstract_lemma'] = df_patstat['title_abstract'].progress_apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

100%|██████████| 696354/696354 [03:12<00:00, 3625.52it/s]


# Build Document-Term Matrix

In [6]:
ngrams = df_cleantech_keywords['keyword_yake_lemma'].tolist()
documents = df_patstat['title_abstract_lemma'].tolist()

In [7]:
def custom_tokenizer(text, ngrams):
    pattern = '|'.join(re.escape(phrase) for phrase in ngrams)
    tokens = []
    last_end = 0
    for match in re.finditer(pattern, text):
        tokens.extend(text[last_end:match.start()].split())
        tokens.append(match.group(0))
        last_end = match.end()
    tokens.extend(text[last_end:].split())
    return tokens

In [8]:
def tokenizer_wrapper(text):
    return custom_tokenizer(text, ngrams)

In [9]:
vectorizer = CountVectorizer(tokenizer=tokenizer_wrapper, vocabulary=ngrams)

In [10]:
patstat_cleantech_dtm = vectorizer.fit_transform(documents)

/home/thiesen/Documents/Cleantech-Network-Analysis/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [26]:
df_patstat_cleantech_dtm = pd.DataFrame(patstat_cleantech_dtm.toarray(),
                                        columns=vectorizer.get_feature_names_out())
df_patstat_cleantech_dtm.reset_index(drop=True, inplace=True)
df_patstat.reset_index(drop=True, inplace=True)
df_patstat_cleantech_dtm.insert(0, 'appln_id', df_patstat['appln_id'])

In [34]:
df_patstat_cleantech_dtm.to_csv('/mnt/hdd01/Cleantech Network Analysis/df_patstat_cleantech_dtm.csv', index=False)